In [1]:
!pip install easyfsl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.8/72.8 kB 1.0 MB/s eta 0:00:00-:--:--


In [2]:
import os

import copy
import pandas as pd
import numpy as np
import torch.nn.functional as F
import torch.utils.data as data

import torch
import torchvision
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import resnet18
from tqdm import tqdm
import torchvision.models
from PIL import Image
from easyfsl.samplers import TaskSampler
from easyfsl.utils import plot_images, sliding_average
import matplotlib.pyplot as plt
from torchvision.io import read_image
import imageio.v2 as imageio
import re
import random
from statistics import mean

from tqdm import tqdm

In [3]:
abnormal_df = pd.read_csv('/kaggle/input/mrnet-v1/MRNet-v1.0/train-abnormal.csv', names=['image_id', 'abnormal'])
acl_df = pd.read_csv('/kaggle/input/mrnet-v1/MRNet-v1.0/train-acl.csv', names=['image_id', 'acl'])
meniscus_df = pd.read_csv('/kaggle/input/mrnet-v1/MRNet-v1.0/train-meniscus.csv', names=['image_id', 'meniscus'])
merged_df = pd.merge(acl_df, meniscus_df, how='inner', on='image_id')
merged_df = pd.merge(merged_df, abnormal_df, how='inner', on='image_id')
merged_df['label'] = merged_df['abnormal'] * 1 + merged_df['meniscus'] * 2 + merged_df['acl'] * 4
merged_df['image_id'] = merged_df['image_id'].map(lambda i: '0' * (4 - len(str(i))) + str(i))
label_mapping = {0: 0, 1: 1, 3: 2, 5: 3, 7: 4}
merged_df['label'] = merged_df['label'].map(label_mapping)
train_df=merged_df

In [4]:
# Label
# 0 - No abnormility, acl or meniscus
# 1 - only abnormility
# 2 - abnormility and meniscus
# 3 - abnormility and acl
# 4 - abnormility and meniscus and acl

In [5]:
train_df.head()

,image_id,acl,meniscus,abnormal,label
0,0000,0,0,1,1
1,0001,1,1,1,4
2,0002,0,0,1,1
3,0003,0,1,1,2
4,0004,0,0,1,1


In [6]:
train_df['label'].value_counts()

label
1    433
2    272
0    217
4    125
3     83
Name: count, dtype: int64

In [7]:

# # Read CSV files and drop rows with label 0
# abnormal_df = pd.read_csv('/kaggle/input/mrnet-v1/MRNet-v1.0/train-abnormal.csv', names=['image_id', 'label']).query('label != 0')
# acl_df = pd.read_csv('/kaggle/input/mrnet-v1/MRNet-v1.0/train-acl.csv', names=['image_id', 'label']).query('label != 0')
# meniscus_df = pd.read_csv('/kaggle/input/mrnet-v1/MRNet-v1.0/train-meniscus.csv', names=['image_id', 'label']).query('label != 0')

# # Assign labels to each DataFrame
# abnormal_df['label'] = 0
# acl_df['label'] = 1
# meniscus_df['label'] = 2

# acl_df['image_id'] = acl_df['image_id'].map(lambda i: '0' * (4 - len(str(i))) + str(i))
# meniscus_df['image_id'] = meniscus_df['image_id'].map(lambda i: '0' * (4 - len(str(i))) + str(i))
# abnormal_df['image_id'] = abnormal_df['image_id'].map(lambda i: '0' * (4 - len(str(i))) + str(i))

# # Merge DataFrames with preference to abnormal and then acl
# train_df = pd.concat([acl_df, meniscus_df, abnormal_df], ignore_index=True, sort=False)

# # Drop duplicates and keep the first occurrence (preference to abnormal and then acl)
# train_df = train_df.drop_duplicates(subset='image_id', keep='first')



In [8]:
# abnormal_df = pd.read_csv('/kaggle/input/mrnet-v1/MRNet-v1.0/valid-abnormal.csv', names=['image_id', 'label']).query('label != 0')
# acl_df = pd.read_csv('/kaggle/input/mrnet-v1/MRNet-v1.0/valid-acl.csv', names=['image_id', 'label']).query('label != 0')
# meniscus_df = pd.read_csv('/kaggle/input/mrnet-v1/MRNet-v1.0/valid-meniscus.csv', names=['image_id', 'label']).query('label != 0')

# # Assign labels to each DataFrame
# abnormal_df['label'] = 0
# acl_df['label'] = 1
# meniscus_df['label'] = 2

# acl_df['image_id'] = acl_df['image_id'].map(lambda i: '0' * (4 - len(str(i))) + str(i))
# meniscus_df['image_id'] = meniscus_df['image_id'].map(lambda i: '0' * (4 - len(str(i))) + str(i))
# abnormal_df['image_id'] = abnormal_df['image_id'].map(lambda i: '0' * (4 - len(str(i))) + str(i))

# # Merge DataFrames with preference to abnormal and then acl
# test_df = pd.concat([acl_df, meniscus_df, abnormal_df], ignore_index=True, sort=False)

# # Drop duplicates and keep the first occurrence (preference to abnormal and then acl)
# test_df = test_df.drop_duplicates(subset='image_id', keep='first')



In [9]:
# train_df['label'].value_counts()

In [10]:
abnormal_df = pd.read_csv('/kaggle/input/mrnet-v1/MRNet-v1.0/valid-abnormal.csv', names=['image_id', 'abnormal'])
acl_df = pd.read_csv('/kaggle/input/mrnet-v1/MRNet-v1.0/valid-acl.csv', names=['image_id', 'acl'])
meniscus_df = pd.read_csv('/kaggle/input/mrnet-v1/MRNet-v1.0/valid-meniscus.csv', names=['image_id', 'meniscus'])
merged_df = pd.merge(acl_df, meniscus_df, how='inner', on='image_id')
merged_df = pd.merge(merged_df, abnormal_df, how='inner', on='image_id')
merged_df['label'] = merged_df['abnormal'] * 1 + merged_df['meniscus'] * 2 + merged_df['acl'] * 4
merged_df['image_id'] = merged_df['image_id'].map(lambda i: '0' * (4 - len(str(i))) + str(i))
label_mapping = {0: 0, 1: 1, 3: 2, 5: 3, 7: 4}
merged_df['label'] = merged_df['label'].map(label_mapping)
test_df=merged_df

In [11]:
test_df.head()

,image_id,acl,meniscus,abnormal,label
0,1130,0,0,0,0
1,1131,0,0,0,0
2,1132,0,0,0,0
3,1133,0,0,0,0
4,1134,0,0,0,0


In [12]:
test_df['label'].value_counts()

label
4    31
0    25
3    23
2    21
1    20
Name: count, dtype: int64

In [13]:
df_train = train_df[train_df['label'].isin([1, 2,4])]


In [14]:
df_train['label'].value_counts()

label
1    433
2    272
4    125
Name: count, dtype: int64

In [15]:
df_test = test_df[test_df['label'].isin([0, 3])]

In [16]:
selected_classes = [0,3]

# Initialize an empty DataFrame for the sampled data
sampled_df = pd.DataFrame()

# Iterate through each selected class and sample 20% of data from each class
for class_label in selected_classes:
    class_data = train_df[train_df['label'] == class_label]
    sampled_class_data = class_data.sample(frac=0.3, random_state=42)  # Adjust the random_state if needed
    sampled_df = pd.concat([sampled_df, sampled_class_data])

In [17]:
df_valid=sampled_df

In [18]:
df_train['label'].value_counts()

label
1    433
2    272
4    125
Name: count, dtype: int64

In [19]:
df_valid['label'].value_counts()

label
0    65
3    25
Name: count, dtype: int64

In [20]:
def interpolate_mri_images(images, num_interpolated_slices=15, interpolation_ratio=0.8):
    
    # Calculate the number of original slices
    num_original_slices = images.shape[0]

    # Calculate the interpolation step size
    interpolation_step = (num_original_slices - 1) / (num_interpolated_slices - 1)

    # Create an array to store the interpolated images
    interpolated_images = []

    # Iterate through each interpolated slice
    for i in range(num_interpolated_slices):
        # Calculate the index of the nearest original slices for interpolation
        base_index = int(i * interpolation_step)
        next_index = min(base_index + 1, num_original_slices - 1)

        # Extract the original slices for interpolation
        base_slice = images[base_index]
        next_slice = images[next_index]

        # Perform linear interpolation
        interpolated_slice = (1 - interpolation_ratio) * base_slice + interpolation_ratio * next_slice

        # Append the interpolated slice to the result
        interpolated_images.append(interpolated_slice)

    # Convert the result to a NumPy array
    interpolated_images = np.array(interpolated_images)

    return interpolated_images

In [21]:
import cv2
def apply_transformations(image_array):
    transformed_frames = []
    for frame in image_array:
        # Resize
        resized_frame = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)

        # Random horizontal flip
        if np.random.rand() < 0.5:
            resized_frame = cv2.flip(resized_frame, 1)

        # Random vertical flip
        if np.random.rand() < 0.5:
            resized_frame = cv2.flip(resized_frame, 0)

        # Random rotation
        angle = np.random.uniform(-45, 45)
        rows, cols = resized_frame.shape
        rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        rotated_frame = cv2.warpAffine(resized_frame, rotation_matrix, (cols, rows))

        # Random affine transformation
        scale = 1.0
        angle = np.random.uniform(-45, 45)
        translation = (np.random.uniform(-0.1, 0.1) * cols, np.random.uniform(-0.1, 0.1) * rows)
        affine_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, scale)
        affine_matrix[:, 2] += translation
        transformed_frame = cv2.warpAffine(rotated_frame, affine_matrix, (cols, rows))

        transformed_frames.append(transformed_frame)
       
    # Convert to tensor
    transformed_frames = np.array(transformed_frames)

    return transformed_frames


In [22]:
import cv2
def apply_transformations(image_array):
    transformed_frames = []
    for frame in image_array:
        # Resize
        resized_frame = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)

        # Random horizontal flip
        if np.random.rand() < 0.5:
            resized_frame = cv2.flip(resized_frame, 1)

        # Random vertical flip
        if np.random.rand() < 0.5:
            resized_frame = cv2.flip(resized_frame, 0)

        # Random rotation
        angle = np.random.uniform(-45, 45)
        rows, cols = resized_frame.shape
        rotation_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, 1)
        rotated_frame = cv2.warpAffine(resized_frame, rotation_matrix, (cols, rows))

        # Random affine transformation
        scale = 1.0
        angle = np.random.uniform(-45, 45)
        translation = (np.random.uniform(-0.1, 0.1) * cols, np.random.uniform(-0.1, 0.1) * rows)
        affine_matrix = cv2.getRotationMatrix2D((cols / 2, rows / 2), angle, scale)
        affine_matrix[:, 2] += translation
        transformed_frame = cv2.warpAffine(rotated_frame, affine_matrix, (cols, rows))

        transformed_frames.append(transformed_frame)
       
    # Convert to tensor
    transformed_frames = np.array(transformed_frames)

    return transformed_frames


In [23]:
def apply_transformations_test(image_array):
    transformed_frames = []
    for frame in image_array:
        # Resize
        resized_frame = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA)

        # Random horizontal flip
        if np.random.rand() < 0.5:
            resized_frame = cv2.flip(resized_frame, 1)

        transformed_frames.append(resized_frame)
       
    # Convert to tensor
    transformed_frames = np.array(transformed_frames)

    return transformed_frames

In [24]:
class MRDataset(data.Dataset):
    def __init__(self, root_dir,plane, train="train" ,transform=None):
        self.plane = plane
        self.transform = transform
        self.train = train
        if self.train=="train":
            self.records = df_train
            self.img_dir = root_dir+ 'train/{0}/'.format(plane)
        elif self.train=="valid":
            self.records = df_valid
            self.img_dir = root_dir+ 'train/{0}/'.format(plane)
        
        else:
            self.records = df_test
            self.img_dir = root_dir+ 'valid/{0}/'.format(plane)
        self.paths = [self.img_dir + filename +
                      '.npy' for filename in self.records['image_id'].tolist()]
        self.labels = self.records['label'].tolist()
        
    def __len__(self):
        return len(self.paths)
    
    def get_labels(self):
        return self.labels
    
    def __getitem__(self, index):
        img_path = self.paths[index]
        image =  np.load(img_path)
        label = self.labels[index]
        image=interpolate_mri_images(image)
        if self.transform:
            if self.train=="train":
                image = apply_transformations(image)
            else:
                image = apply_transformations_test(image)
        image = (image - 58.09) / 49.73
        image = np.stack((image,)*1, axis=3)
        image = torch.FloatTensor(image)
        image=image.reshape(image.shape[3],image.shape[0],image.shape[1],image.shape[2])
        return image,label

In [25]:
root_dir = "/kaggle/input/mrnet-v1/MRNet-v1.0/"

In [26]:
mrnet_dataset = MRDataset(root_dir,plane="axial",train="train",transform=True)

# Create a DataLoader for your dataset
batch_size = 1
dataloader = DataLoader(dataset=mrnet_dataset, batch_size=batch_size, shuffle=True)

In [27]:
test_features, test_labels = next(iter(dataloader))
print(f"Feature batch shape: {test_features.size()}")
print(f"Labels batch shape: {test_labels.size()}")

Feature batch shape: torch.Size([1, 1, 15, 224, 224])
Labels batch shape: torch.Size([1])


In [28]:
mrnet_dataset = MRDataset(root_dir,plane="axial",train="valid",transform=True)

# Create a DataLoader for your dataset
batch_size = 1
dataloader = DataLoader(dataset=mrnet_dataset, batch_size=batch_size, shuffle=True)

In [29]:
test_features, test_labels = next(iter(dataloader))
print(f"Feature batch shape: {test_features.size()}")
print(f"Labels batch shape: {test_labels.size()}")

Feature batch shape: torch.Size([1, 1, 15, 224, 224])
Labels batch shape: torch.Size([1])


## Training a Meta Trainer

In [30]:
root_dir = "/kaggle/input/mrnet-v1/MRNet-v1.0/"

In [31]:
train_set= MRDataset(root_dir,plane="axial",train="train", transform=True)

In [32]:
val_set = MRDataset(root_dir,plane="axial",train="valid", transform=True)

In [33]:
n_tasks_per_epoch = 500
n_validation_tasks = 100

train_sampler = TaskSampler(
    train_set, n_way=3, n_shot=5, n_query=10, n_tasks=n_tasks_per_epoch
)
train_loader = DataLoader(
    train_set,
    batch_sampler=train_sampler,
    num_workers=4,
    pin_memory=True,
    collate_fn=train_sampler.episodic_collate_fn,
)

In [35]:
val_sampler = TaskSampler(
    val_set, n_way=2, n_shot=5, n_query=10, n_tasks=n_validation_tasks
)
val_loader = DataLoader(
    val_set,
    batch_sampler=val_sampler,
    num_workers=4,
    pin_memory=True,
    collate_fn=val_sampler.episodic_collate_fn,
)

In [ ]:
# from torchvision.models import video


In [ ]:
# class PrototypicalNetworks(nn.Module):
#     def __init__(self, backbone: nn.Module):
#         super(PrototypicalNetworks, self).__init__()
#         self.backbone = backbone

#     def forward(
#         self,
#         support_images: torch.Tensor,
#         support_labels: torch.Tensor,
#         query_images: torch.Tensor,
#     ) -> torch.Tensor:
#         """
#         Predict query labels using labeled support images.
#         """
#         # Extract the features of support and query images
#         z_support = self.backbone.forward(support_images)
#         z_query = self.backbone.forward(query_images)

#         # Infer the number of different classes from the labels of the support set
#         n_way = len(torch.unique(support_labels))
#         # Prototype i is the mean of all instances of features corresponding to labels == i
#         z_proto = torch.cat(
#             [
#                 z_support[torch.nonzero(support_labels == label)].mean(0)
#                 for label in range(n_way)
#             ]
#         )

#         # Compute the euclidean distance from queries to prototypes
#         dists = torch.cdist(z_query, z_proto)

#         # And here is the super complicated operation to transform those distances into classification scores!
#         scores = -dists
#         return scores


# convolutional_network = video.r3d_18(pretrained=True)

# convolutional_network.fc = nn.Flatten()
# convolutional_network = nn.DataParallel(convolutional_network)
# print(convolutional_network)

# model = PrototypicalNetworks(convolutional_network).cuda()

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import math
from functools import partial

__all__ = [
    'ResNet', 'resnet10', 'resnet18', 'resnet34', 'resnet50', 'resnet101',
    'resnet152', 'resnet200'
]


def conv3x3x3(in_planes, out_planes, stride=1, dilation=1):
    # 3x3x3 convolution with padding
    return nn.Conv3d(
        in_planes,
        out_planes,
        kernel_size=3,
        dilation=dilation,
        stride=stride,
        padding=dilation,
        bias=False)


def downsample_basic_block(x, planes, stride, no_cuda=False):
    out = F.avg_pool3d(x, kernel_size=1, stride=stride)
    zero_pads = torch.Tensor(
        out.size(0), planes - out.size(1), out.size(2), out.size(3),
        out.size(4)).zero_()
    if not no_cuda:
        if isinstance(out.data, torch.cuda.FloatTensor):
            zero_pads = zero_pads.cuda()

    out = Variable(torch.cat([out.data, zero_pads], dim=1))

    return out


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, dilation=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3x3(inplanes, planes, stride=stride, dilation=dilation)
        self.bn1 = nn.BatchNorm3d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3x3(planes, planes, dilation=dilation)
        self.bn2 = nn.BatchNorm3d(planes)
        self.downsample = downsample
        self.stride = stride
        self.dilation = dilation

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, dilation=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv3d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm3d(planes)
        self.conv2 = nn.Conv3d(
            planes, planes, kernel_size=3, stride=stride, dilation=dilation, padding=dilation, bias=False)
        self.bn2 = nn.BatchNorm3d(planes)
        self.conv3 = nn.Conv3d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm3d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.dilation = dilation

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class upsample(nn.Module):
    def forward(self, inp):
        return F.interpolate(inp, scale_factor = 2)

class ResNet(nn.Module):

    def __init__(self,
                 block,
                 layers,
                 num_seg_classes=2,
                 shortcut_type='B',
                 no_cuda = False):
        self.inplanes = 64
        self.no_cuda = no_cuda
        super(ResNet, self).__init__()
        self.conv1 = nn.Conv3d(
            1,
            64,
            kernel_size=7,
            stride=(2, 2, 2),
            padding=(3, 3, 3),
            bias=False)
            
        self.bn1 = nn.BatchNorm3d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool3d(kernel_size=(3, 3, 3), stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], shortcut_type)
        self.layer2 = self._make_layer(
            block, 128, layers[1], shortcut_type, stride=2)
        self.layer3 = self._make_layer(
            block, 256, layers[2], shortcut_type, stride=1, dilation=2)
        self.layer4 = self._make_layer(
            block, 512, layers[3], shortcut_type, stride=1, dilation=4)

        self.conv_seg = nn.Sequential(
                                        upsample(),
                                        nn.ConvTranspose3d(
                                        512 * block.expansion,
                                        32,
                                        2,
                                        stride=2
                                        ),
                                        nn.BatchNorm3d(32),
                                        nn.ReLU(inplace=True),
                                        upsample(),
                                        nn.Conv3d(
                                        32,
                                        32,
                                        kernel_size=3,
                                        stride=(1, 1, 1),
                                        padding=(1, 1, 1),
                                        bias=False), 
                                        nn.BatchNorm3d(32),
                                        nn.ReLU(inplace=True),
                                        nn.Conv3d(
                                        32,
                                        num_seg_classes,
                                        kernel_size=1,
                                        stride=(1, 1, 1),
                                        bias=False),
                                        nn.Sigmoid() 
                                        )

        for m in self.modules():
            if isinstance(m, nn.Conv3d):
                m.weight = nn.init.kaiming_normal_(m.weight, mode='fan_out')
            elif isinstance(m, nn.BatchNorm3d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, shortcut_type, stride=1, dilation=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            if shortcut_type == 'A':
                downsample = partial(
                    downsample_basic_block,
                    planes=planes * block.expansion,
                    stride=stride,
                    no_cuda=self.no_cuda)
            else:
                downsample = nn.Sequential(
                    nn.Conv3d(
                        self.inplanes,
                        planes * block.expansion,
                        kernel_size=1,
                        stride=stride,
                        bias=False), nn.BatchNorm3d(planes * block.expansion))

        layers = []
        layers.append(block(self.inplanes, planes, stride=stride, dilation=dilation, downsample=downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes, dilation=dilation))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        #print(x.shape)
        x = self.conv_seg(x)

        return x

def resnet10(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = ResNet(BasicBlock, [1, 1, 1, 1], **kwargs)
    return model


def resnet18(**kwargs):
    """Constructs a ResNet-18 model.
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    return model


def resnet34(**kwargs):
    """Constructs a ResNet-34 model.
    """
    model = ResNet(BasicBlock, [3, 4, 6, 3], **kwargs)
    return model


def resnet50(**kwargs):
    """Constructs a ResNet-50 model.
    """
    model = ResNet(Bottleneck, [3, 4, 6, 3], **kwargs)
    return model


def resnet101(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 4, 23, 3], **kwargs)
    return model


def resnet152(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 8, 36, 3], **kwargs)
    return model


def resnet200(**kwargs):
    """Constructs a ResNet-101 model.
    """
    model = ResNet(Bottleneck, [3, 24, 36, 3], **kwargs)
    return model

In [37]:
def trim_state_dict_name(state_dict):
    for k in list(state_dict.keys()):
        if k.startswith('module.'):
            # remove prefix
            state_dict[k[len("module."):]] = state_dict[k]
            del state_dict[k]
    return state_dict

In [38]:
def get_feature_extractor():
    model = resnet50(shortcut_type='B')
    model.conv_seg = nn.Flatten()
    # download pretrained weight from https://drive.google.com/file/d/13tnSvXY7oDIEloNFiGTsjUIYfS3g3BfG/view?usp=sharing
    ckpt = torch.load("/kaggle/input/resnet50-add/resnet_50_23dataset.pth")
    ckpt = trim_state_dict_name(ckpt["state_dict"])
    model.load_state_dict(ckpt)
    model = nn.DataParallel(model).cuda()
    model.eval()
    print("Feature extractor weights loaded")
    return model

model = get_feature_extractor()

Feature extractor weights loaded


In [39]:
from easyfsl.methods import PrototypicalNetworks, FewShotClassifier
convolutional_network = model
few_shot_classifier = PrototypicalNetworks(convolutional_network).cuda()

In [40]:
# class PrototypicalNetworks(nn.Module):
#     def __init__(self, backbone: nn.Module):
#         super(PrototypicalNetworks, self).__init__()
#         self.backbone = backbone

#     def forward(
#         self,
#         support_images: torch.Tensor,
#         support_labels: torch.Tensor,
#         query_images: torch.Tensor,
#     ) -> torch.Tensor:
#         """
#         Predict query labels using labeled support images.
#         """
#         # Extract the features of support and query images
#         z_support = self.backbone.forward(support_images)
#         z_query = self.backbone.forward(query_images)

#         # Infer the number of different classes from the labels of the support set
#         n_way = len(torch.unique(support_labels))
#         # Prototype i is the mean of all instances of features corresponding to labels == i
#         z_proto = torch.cat(
#             [
#                 z_support[torch.nonzero(support_labels == label)].mean(0)
#                 for label in range(n_way)
#             ]
#         )

#         # Compute the euclidean distance from queries to prototypes
#         dists = torch.cdist(z_query, z_proto)

#         # And here is the super complicated operation to transform those distances into classification scores!
#         scores = -dists
#         return scores


# convolutional_network = model
# print(convolutional_network)

# model = PrototypicalNetworks(convolutional_network).cuda()


In [41]:
from torch.optim import SGD, Optimizer
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from pathlib import Path

LOSS_FUNCTION = nn.CrossEntropyLoss()

n_epochs = 10
scheduler_milestones = [120, 160]
scheduler_gamma = 0.1
learning_rate = 1e-2
tb_logs_dir = Path(".")

train_optimizer = SGD(
    few_shot_classifier.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4
)
train_scheduler = MultiStepLR(
    train_optimizer,
    milestones=scheduler_milestones,
    gamma=scheduler_gamma,
)

tb_writer = SummaryWriter(log_dir=str(tb_logs_dir))

2024-02-01 04:52:08.893635: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-01 04:52:08.893743: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-01 04:52:09.057058: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [42]:
def training_epoch(
    model: FewShotClassifier, data_loader: DataLoader, optimizer: Optimizer
):
    all_loss = []
    model.train()
    with tqdm(
        enumerate(data_loader), total=len(data_loader), desc="Training"
    ) as tqdm_train:
        for episode_index, (
            support_images,
            support_labels,
            query_images,
            query_labels,
            _,
        ) in tqdm_train:
            optimizer.zero_grad()
            model.process_support_set(
                support_images.cuda(), support_labels.cuda()
            )
            classification_scores = model(query_images.cuda())

            loss = LOSS_FUNCTION(classification_scores, query_labels.cuda())
            loss.backward()
            optimizer.step()

            all_loss.append(loss.item())

            tqdm_train.set_postfix(loss=mean(all_loss))

    return mean(all_loss)

In [61]:
from easyfsl.utils import evaluate


best_state = few_shot_classifier.state_dict()
best_validation_accuracy = 0.0
for epoch in range(n_epochs):
    print(f"Epoch {epoch}")
    average_loss = training_epoch(few_shot_classifier, train_loader, train_optimizer)
    validation_accuracy = evaluate(
        few_shot_classifier, val_loader, device="cuda", tqdm_prefix="Validation"
    )

    if validation_accuracy > best_validation_accuracy:
        best_validation_accuracy = validation_accuracy
        best_state = copy.deepcopy(few_shot_classifier.state_dict())
        # state_dict() returns a reference to the still evolving model's state so we deepcopy
        # https://pytorch.org/tutorials/beginner/saving_loading_models
        print("Ding ding ding! We found a new best model!")

    tb_writer.add_scalar("Train/loss", average_loss, epoch)
    tb_writer.add_scalar("Val/acc", validation_accuracy, epoch)

    # Warn the scheduler that we did an epoch
    # so it knows when to decrease the learning rate
    train_scheduler.step()

Epoch 0


Training:   0%|          | 0/500 [00:05<?, ?it/s]


KeyboardInterrupt: 

In [62]:
few_shot_classifier.load_state_dict(best_state)

<All keys matched successfully>

## Testing

In [63]:
df_test = test_df[test_df['label'].isin([0, 3])]

In [64]:
df_test['label'].value_counts()

label
0    25
3    23
Name: count, dtype: int64

In [65]:
root_dir = "/kaggle/input/mrnet-v1/MRNet-v1.0/"

In [66]:
test_set= MRDataset(root_dir,plane="axial",train=False)

In [67]:
N_WAY = 2  # Number of classes in a task
N_SHOT = 8  # Number of images per class in the support set
N_QUERY = 10  # Number of images per class in the query set
N_EVALUATION_TASKS = 100

# The sampler needs a dataset with a "get_labels" method. Check the code if you have any doubt!

test_sampler = TaskSampler(
    test_set, n_way=N_WAY, n_shot=N_SHOT, n_query=N_QUERY, n_tasks=N_EVALUATION_TASKS
)

test_loader = DataLoader(
    test_set,
    batch_sampler=test_sampler,
    num_workers=4,
    pin_memory=True,
    collate_fn=test_sampler.episodic_collate_fn,
)

In [69]:
accuracy = evaluate(few_shot_classifier, test_loader)
print(f"Average accuracy : {(100 * accuracy):.2f} %")

100%|██████████| 100/100 [02:54<00:00,  1.75s/it, accuracy=0.81]

Average accuracy : 81.00 %
